In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
np.random.seed(0)

In [2]:
data = pd.read_csv("mnist_train.csv")
test = pd.read_csv("mnist_test.csv")

In [3]:
test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
data = np.array(data)
test = np.array(test)
m,n = data.shape
labels = data[:,0]
values = data[:,1:n]/255
test_labels = test[:,0]
test_values = test[:,1:n]

In [127]:
def oneHotEncode(len,value):
    vector = np.zeros((len,1))
    vector[value,0] = 1
    return vector

def checkAccuracy(prediction,value):
    return prediction.argmax() == value

def calculateCategoricalCrossEntropyLoss(prediction,true):
    y_pred_clipped = np.clip(prediction,1e-7,1-1e-7)
    return -np.log(np.sum(y_pred_clipped*oneHotEncode(10,true)))

def calculateError(prediction,true):
    return prediction - oneHotEncode(10,true)

In [149]:

class Layer:
    def __init__(self):
        self.inputNum = None
        self.outputNum = None

class Dense(Layer):
    def __init__(self,n_inputs,n_neurons):
        self.weights = np.random.rand(n_neurons,n_inputs) -0.5
        self.bias = np.zeros((n_neurons,1))

    def foward(self, input):
        self.inputs = input
        return np.dot(self.weights,self.inputs) + self.bias
    
    def backward(self,gradient,learningRate):
        weights_gradient = np.dot(gradient,self.inputs.T)
        self.weights -= learningRate*weights_gradient
        self.bias -= learningRate*gradient
        return np.dot(self.weights.T,gradient)
    

class NeuralNetwork():
    def __init__(self,layers):
        self.layers = layers

    def fowardStep(self,input):
        inputData = input
        for layer in self.layers:
            inputData = layer.foward(inputData)
        return inputData
    
    def backwardStep(self,gradient,learningRate):
        gradientData = gradient
        for layer in reversed(self.layers):
            gradientData = layer.backward(gradientData,learningRate)
        return gradientData


    def train(self,images,labels,epochs,learningRate):
        for epoch in range(epochs):
            correctGuesses = 0
            for img,label in zip(images,labels):
                currImg = np.reshape(img,(n-1,1))
                fowardResult = self.fowardStep(currImg)
                correctGuesses += 1 if checkAccuracy(fowardResult,label) else 0
                gradient = calculateError(fowardResult,label)
                self.backwardStep(gradient,learningRate)
            print(f"Epoch {epoch} --> Accuracy = {correctGuesses/len(labels)*100:.2f}%")




In [151]:
class Activation_Relu:
    def foward(self,inputs):
        self.inputs = inputs
        self.outputs = np.maximum(0,inputs)
        return self.outputs
    
    def backward(self,gradient,learningRate):
        return np.multiply(gradient,(self.inputs > 0).astype(float))
    
class Activation_Softmax:
    def foward(self,inputs):
        self.inputs = inputs
        normalizedInput = inputs-np.max(inputs)
        tmp = np.exp(normalizedInput)
        self.output = tmp/np.sum(tmp)
        return self.output
    def backward(self,gradient,learningRate):
        return gradient


In [152]:
network = NeuralNetwork([
    Dense(784,10),
    Activation_Relu(),
    Dense(10,10),
    Activation_Softmax()
])

In [153]:
network.train(values,labels,5,0.01)

Epoch 0 --> Accuracy = 20.07%


C:\Users\savic\AppData\Local\Temp\ipykernel_5176\1791772533.py:13: RuntimeWarning: overflow encountered in subtract
  normalizedInput = inputs-np.max(inputs)
C:\Users\savic\AppData\Local\Temp\ipykernel_5176\1791772533.py:13: RuntimeWarning: invalid value encountered in subtract
  normalizedInput = inputs-np.max(inputs)


Epoch 1 --> Accuracy = 17.79%
Epoch 2 --> Accuracy = 9.87%


KeyboardInterrupt: 

In [69]:
img = np.reshape(values[0],(n-1,1))
network.fowardStep(img)
dense = Dense(784,10)
relu = Activation_Relu()
dense.foward(img)
relu.foward(np.array([[1362.3602796 ],
       [-1.5292648 ],
       [1347.32393459],
       [1468.91357089],
       [1318.23949071],
       [1211.72654485],
       [1450.41173214],
       [1459.56618337],
       [1430.52703296],
       [1382.73239906]]))
relu.backward()

array([[1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [121]:
np.random.rand(5,5)

array([[0.15384315, 0.84404908, 0.82154751, 0.11424367, 0.66304221],
       [0.12791221, 0.72052166, 0.61822103, 0.37649339, 0.92353805],
       [0.68126916, 0.95216415, 0.45247457, 0.6524256 , 0.61162184],
       [0.23525591, 0.18636773, 0.67900856, 0.97000427, 0.0453332 ],
       [0.22669474, 0.56373332, 0.83214681, 0.04454144, 0.91967361]])